In [69]:
################################################################################
from typing import Sequence

class Calc:
    def __init__(self,
            symbols : Sequence[str] = ("0", "1"),
            ) -> None:
        self.symbols = symbols
        self.next = dict(
            (symbols[i], symbols[(i + 1) % len(symbols)])
            for i in range(len(symbols))
        )
    
    def increment(self, string: str) -> str:
        assert(all((i in self.symbols) for i in string))
        for index in range((len(string) - 1), -1, -1):
            if string[index] != self.symbols[-1]:
                return (
                    string[:index]
                    + self.next[string[index]]
                    + (len(string) - index - 1) * self.symbols[0]
                )
        return self.symbols[1] + len(string) * self.symbols[0]
    
    def eq(self, string_1: str, string_2: str) -> str:
        if len(string_1) != len(string_2):
            return "0"
        for i in range(len(string_1)):
            if string_1[i] != string_2[i]:
                return "0"
        return "1"
    
    def lt(self, string_1: str, string_2: str) -> str:
        
    
    def add(self, string_1: str, string_2: str) -> str:
        assert(all(i in self.symbols) for i in string_1)
        assert(all(i in self.symbols) for i in string_2)
        counter = self.symbols[0]
        while counter != 
        
c = Calc(symbols=("0", "1", "2", "3", "4", "5", "6", "7", "8", "9"))
print(c.increment("150"))


151


In [1]:
!nvidia-smi

Mon Nov 20 15:48:58 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:04:00.0 Off |                  N/A |
| 34%   51C    P2    52W / 180W |   1022MiB /  8114MiB |     18%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 00000000:05:00.0 Off |                  N/A |
| 34%   51C    P2    47W / 180W |    819MiB /  8114MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
### Imports ####################################################################
%matplotlib inline
import os
import numpy as np
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import matplotlib.pyplot as plt
import theano
import theano.tensor as T
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, GRU
from typing import Tuple, Callable, List, Optional
from tracker import visuals, extractor, utils, metrics

np.set_printoptions(suppress=True)
Tensor = theano.tensor.Tensor

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:0C:00.0)
Using Theano backend.


In [70]:
### Load in Data ###############################################################
order  = ["phi", "r", "z"]
frame  = pd.read_csv("data/sets/ACTS-MU10-PT1000-T50-PREPARED.gz")
data   = extractor.extract_input(frame, order)
matrix = extractor.extract_output(frame, order)
input_shape  = data.shape[1:]
output_shape = matrix.shape[1:]
print(len(data))
print(input_shape)
print(output_shape)
n = 3
visuals.display_matrices(data[n], matrix[n], order=order, noise=False)

65407
(200, 3)
(200, 52)


,phi,r,z,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,[,\,],^,_,`,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,padding
0,1.471408,31.959895,-192.97200,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.483444,71.842967,-371.65000,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,2.500341,71.842967,-432.62400,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,2.508099,31.959895,-219.80000,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,2.665143,31.959895,-31.34380,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,2.673906,71.842967,-89.13750,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,2.683948,115.818695,-155.34100,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,2.696099,171.809967,-235.49400,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,2.991512,31.959895,7.66250,,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,2.998506,71.842967,81.51560,,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [71]:
### Run this cell if you want all events to have max number of hits ############
def regularize_frame(
        frame      : pd.DataFrame,
        num_layers : int,  # How many layers (hits per track).
        num_tracks : int,  # How many tracks per event.
        ) -> pd.DataFrame:
    regular = []
    events = (e for _, e in frame[frame["padding"] == 0].groupby("event_id"))
    for event in events:
        groups = event.groupby("cluster_id")
        tracks = groups.filter(lambda track: num_layers == len(track))
        if num_tracks <= len(pd.unique(tracks["cluster_id"])):
            ids    = tracks["cluster_id"]
            unique = np.sort(pd.unique(ids))
            mapped = dict((v, u) for (u, v) in enumerate(unique))
            tracks = tracks.assign(cluster_id=ids.map(mapped))
            tracks = tracks[tracks["cluster_id"] < num_tracks]
            if num_tracks == len(pd.unique(tracks["cluster_id"])):
                regular.append(tracks)
    return pd.concat(regular)

frame  = regularize_frame(frame, 4, 5)
data   = extractor.extract_input(frame, order)
matrix = extractor.extract_output(frame, order)
input_shape  = data.shape[1:]
output_shape = matrix.shape[1:]
print(input_shape)
print(output_shape)
n = 2
visuals.display_matrices(data[5], matrix[5], order=order, noise=False, padding=False)

(20, 3)
(20, 5)


,phi,r,z,A,B,C,D,E
0,0.359041,31.959895,-43.811200,1,,,,
1,0.364652,71.842967,-2.973750,1,,,,
2,0.370255,115.818695,41.352800,1,,,,
3,0.377614,171.809967,99.881200,1,,,,
4,0.399905,171.809967,10.076600,,1,,,
5,0.411684,115.818695,3.915560,,1,,,
6,0.420956,71.842967,-0.960523,,1,,,
7,0.429232,31.959895,-5.496880,,1,,,
8,0.460835,171.809967,33.715600,,,1,,
9,0.470633,115.818695,19.653100,,,1,,


In [72]:
#### LossFunctionCreator #######################################################
class LossFunctionCreator:
    def __init__(self,
            input_tensor : Tensor,
            input_shape  : Tuple,
            output_shape : Tuple,
            order        : List[str],
            ) -> None:
        """ Initialize the instance variables. """
        self.__name__     = "LossFunctionCreator"
        self.input_tensor = T.as_tensor_variable(input_tensor)
        self.meshgrid     = self.make_meshgrid(output_shape)
        self.input_shape  = input_shape
        self.output_shape = output_shape
        self.order        = order

    def make_meshgrid(self,
            shape : Tuple[int, int],
            ) -> Tensor:
        """
        Create a meshgrid.
        Example for shape (3, 5):
        [[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]]
        """
        return (T.mgrid[0:shape[0], 0:shape[1]][1])
    
    def combine_with_input_tensor(self,
            input_tensor  : Tensor,
            output_tensor : Tensor,
            ) -> Tensor:
        """
        Return the cartesian product of the input_tensor and the
        output_tensor along the last two axes.
        """
        input_list   = [input_tensor  for _ in range(self.output_shape[-1])]
        output_list  = [output_tensor for _ in range(self.input_shape[-1])]
        input_stack  = T.stack(input_list,  axis=-1)
        output_stack = T.stack(output_list, axis=-2)
        return input_stack * output_stack
    
    def combo_loss(self,
            y_true : Tensor,
            y_pred : Tensor,
            ) -> Tensor:
        """ A loss function. Temporary. """
        input_tensor = self.to_std_dist(self.input_tensor, axis=1)
        true_combo   = self.combine_with_input_tensor(input_tensor, y_true)
        pred_combo   = self.combine_with_input_tensor(input_tensor, y_pred)
        diff_squared = (pred_combo - true_combo)**2
        return diff_squared
    
    def regression_loss(self,
            y_true : Tensor,
            y_pred : Tensor,
            ) -> Tensor:
        input_tensor = self.input_tensor
        # input_tensor = self.to_std_dist(self.input_tensor, axis=1)
        tensor = None
        for i in range(output_shape[1]):  # For each track...
            pred_mask = self.get_track_mask(y_pred, i, aref=256, mref=128)
            true_mask = self.get_track_mask(y_true, i, aref=256, mref=128)
            pred_num_hits = pred_mask.sum(-1).sum(-1)  # Avoid Div(0).
            true_num_hits = true_mask.sum(-1).sum(-1)  # Avoid Div(0).
            pred_masked = pred_mask * input_tensor
            true_masked = true_mask * input_tensor
            pred_line = self.linear_regression(pred_masked, pred_num_hits)
            true_line = self.linear_regression(true_masked, true_num_hits)
            diff   = (pred_line - true_line)**2
            tensor = diff if tensor is None else tensor + diff
        return tensor
    
    def to_std_dist(self,
            tensor : Tensor,
            axis   : int,
            ) -> Tensor:
        """
        Transform the tensor's values to be the number of standard deviations
        that the value is from the mean along the specified axis.
        """
        mean = tensor.mean(axis=axis, keepdims=True)
        std  = tensor.std(axis=axis, keepdims=True)
        return (tensor - mean) / std
    
    def softmax(self,
            tensor     : Tensor,
            axis       : Optional[int] = None,
            refinement : float = 1,
            ) -> Tensor:
        """
        Return the softmax of the tensor along the specified axis.
        Higher refinement yields sharper, more accurate results, but also
        tends to yield NaNs for large tensor values.
        """
        exponent = (refinement * tensor).exp()
        return exponent / exponent.sum(axis=axis, keepdims=True)

    def softargmax(self,
            tensor     : Tensor,
            indices    : Tensor,
            axis       : Optional[int] = None,
            refinement : float = 1,
            ) -> Tensor:
        """
        Return the argsoftmax of the tensor along the specified axis.
        Higher refinement yields sharper, more accurate results, but also
        tends to yield NaNs for large tensor values.
        """
        return (self.softmax(tensor, axis, refinement) * indices).sum(axis)
    
    def get_order_mask(self,
            string : str,
            ) -> Tensor:
        """
        Return a mask such that when output is multiplied by this mask,
        only the column corresponding to the *string* category remains.
        """
        mask = np.zeros(len(self.order))
        mask[self.order.index(string)] = 1
        return T.as_tensor_variable(mask)
    
    def get_track_mask(self,
            output   : Tensor,
            track_id : int,
            aref     : int = 32,  # Refinement value for softargmax.
            mref     : int =  4,  # Refinement value for mask values.
            ) -> Tensor:
        """
        Retrieve a tensor containing a mask such that if self.tensor_input
        was multiplied by the mask, the result would be a tensor containing
        the positions of all hits with the specified track_id.
        """
        cats = self.softargmax(output, self.meshgrid, refinement=aref, axis=-1)
        fill = T.fill(cats, track_id)
        diff = (cats - fill)**2
        mask = 1 / (mref * diff).exp()
        mask = mask.reshape((*T.shape(mask), 1))
        return mask
    
    def linear_regression(self,
            tensor : Tensor,
            length : Tensor,
            ) -> Tensor:
        """
        Given a tensor, and the number of hits within the tensor,
        return the two parameters (m, b) of the least squares
        regression line with equation f[x] = (m * x) + b.
        """
        e = 2 * K.common.epsilon()  # Epsilon to avoid division by 0.
        r = (tensor * self.get_order_mask("r")).sum(-1)  # r values.
        z = (tensor * self.get_order_mask("z")).sum(-1)  # z values.
        d = (length * (z**2).sum(-1)) - z.sum(-1)**2 + e  # Denominator.
        m = (length * (r * z).sum(-1) - z.sum(-1) * r.sum(-1)) / d
        b = (r.sum(-1) * (z**2).sum(-1) - z.sum(-1) * (r * z).sum(-1)) / d
        return m #T.stack([m, b]).T
    
    def __call__(self,
            y_true : Tensor,
            y_pred : Tensor,
            ) -> Tensor:
        """
        Return a Tensor that measures the loss of a model.
        To be used during keras loss compilation.
        """
        return self.regression_loss(y_true, y_pred)

### Define the loss function.

In [73]:
#### Define the loss function ##################################################
A = T.dtensor3("A")
B = T.dtensor3("B")
C = T.dtensor3("C")
D = LossFunctionCreator(A, input_shape, output_shape, order)
E = D(B, C)
F = theano.function([A, B, C], E, on_unused_input='ignore')

In [74]:
################################################################################
def regression(
        data     : np.ndarray, 
        matrix   : np.ndarray,
        order    : List[str],
        track_id : int,
        ) -> Tuple[np.ndarray, np.ndarray]:
    track = data[matrix.argmax(-1) == track_id]
    size = len(track)
    r = track[:, order.index("r")]  # y
    z = track[:, order.index("z")]  # x
    denominator = size * (z**2).sum() - z.sum()**2
    intercept_numerator = r.sum() * (z**2).sum() - z.sum() * (r * z).sum()
    slope_numerator = size * (r * z).sum() - z.sum() * r.sum()
    intercept = intercept_numerator / denominator
    slope = slope_numerator / denominator
    return slope, intercept

def reg_loss(
        positions : np.ndarray, 
        y_true    : np.ndarray, 
        y_pred    : np.ndarray, 
        order     : List[str],
        ) -> np.ndarray:
    loss = 0
    for track_id in range(y_pred.shape[1]):
        true_slope, _ = regression(positions, y_true, order, track_id)
        pred_slope, _ = regression(positions, y_pred, order, track_id)
        loss += (true_slope - pred_slope)**2
    return loss

In [75]:
#### True == Prediction ########################################################
positions   = data  [0:1]
true_matrix = matrix[0:1]
pred_matrix = matrix[0:1]
evaluation  = F(positions, true_matrix, pred_matrix).round(2)
by_hand     = reg_loss(positions[0], true_matrix[0], pred_matrix[0], order) 
print("Tensor Evaluation {}".format(evaluation))
print("By Hand Calculation {}".format(by_hand))
visuals.display_matrices(data[0], true_matrix[0], 2, order, 0, 0)
visuals.display_matrices(data[0], pred_matrix[0], 2, order, 0, 0)

Tensor Evaluation [ 0.]
By Hand Calculation 0.0


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,1,,,,
1,0.272315,115.818695,217.9330,1,,,,
2,0.278982,71.842967,126.6710,1,,,,
3,0.285327,31.959895,42.4250,1,,,,
4,0.623917,31.959895,-65.9719,,1,,,
5,0.634346,71.842967,-120.6220,,1,,,
6,0.645621,115.818695,-180.6470,,1,,,
7,0.659898,171.809967,-256.4190,,1,,,
8,2.950621,31.959895,-47.8166,,,1,,
9,2.956995,71.842967,-78.4979,,,1,,


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,1,,,,
1,0.272315,115.818695,217.9330,1,,,,
2,0.278982,71.842967,126.6710,1,,,,
3,0.285327,31.959895,42.4250,1,,,,
4,0.623917,31.959895,-65.9719,,1,,,
5,0.634346,71.842967,-120.6220,,1,,,
6,0.645621,115.818695,-180.6470,,1,,,
7,0.659898,171.809967,-256.4190,,1,,,
8,2.950621,31.959895,-47.8166,,,1,,
9,2.956995,71.842967,-78.4979,,,1,,


In [9]:
#### Single Roll ###############################################################
positions   = data  [0:1]
true_matrix = matrix[0:1]
pred_matrix = np.roll(matrix[0:1], shift=1, axis=-1)
evaluation  = F(positions, true_matrix, pred_matrix).round(2)
by_hand     = reg_loss(positions[0], true_matrix[0], pred_matrix[0], order) 
print("Tensor Evalution {}".format(evaluation))
print("By Hand Calculation {}".format(by_hand))
visuals.display_matrices(data[0], true_matrix[0], 2, order, 0, 0)
visuals.display_matrices(data[0], pred_matrix[0], 2, order, 0, 0)

Tensor Evalution [ 4.72]
By Hand Calculation 4.715477659383326


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,1,,,,
1,0.272315,115.818695,217.9330,1,,,,
2,0.278982,71.842967,126.6710,1,,,,
3,0.285327,31.959895,42.4250,1,,,,
4,0.623917,31.959895,-65.9719,,1,,,
5,0.634346,71.842967,-120.6220,,1,,,
6,0.645621,115.818695,-180.6470,,1,,,
7,0.659898,171.809967,-256.4190,,1,,,
8,2.950621,31.959895,-47.8166,,,1,,
9,2.956995,71.842967,-78.4979,,,1,,


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,,1,,,
1,0.272315,115.818695,217.9330,,1,,,
2,0.278982,71.842967,126.6710,,1,,,
3,0.285327,31.959895,42.4250,,1,,,
4,0.623917,31.959895,-65.9719,,,1,,
5,0.634346,71.842967,-120.6220,,,1,,
6,0.645621,115.818695,-180.6470,,,1,,
7,0.659898,171.809967,-256.4190,,,1,,
8,2.950621,31.959895,-47.8166,,,,1,
9,2.956995,71.842967,-78.4979,,,,1,


In [10]:
positions   = data  [0:1]
true_matrix = matrix[0:1]
pred_matrix = np.roll(matrix[0:1], shift=5, axis=1)
evaluation  = F(positions, true_matrix, pred_matrix).round(2)
by_hand     = reg_loss(positions[0], true_matrix[0], pred_matrix[0], order) 
print("Tensor Evalution {}".format(evaluation))
print("By Hand Calculation {}".format(by_hand))
visuals.display_matrices(data[0], true_matrix[0], 2, order, 0, 0)
visuals.display_matrices(data[0], pred_matrix[0], 2, order, 0, 0)

Tensor Evalution [ 5.49]
By Hand Calculation 5.491446774145842


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,1,,,,
1,0.272315,115.818695,217.9330,1,,,,
2,0.278982,71.842967,126.6710,1,,,,
3,0.285327,31.959895,42.4250,1,,,,
4,0.623917,31.959895,-65.9719,,1,,,
5,0.634346,71.842967,-120.6220,,1,,,
6,0.645621,115.818695,-180.6470,,1,,,
7,0.659898,171.809967,-256.4190,,1,,,
8,2.950621,31.959895,-47.8166,,,1,,
9,2.956995,71.842967,-78.4979,,,1,,


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,,,,1,
1,0.272315,115.818695,217.9330,,,,,1
2,0.278982,71.842967,126.6710,,,,,1
3,0.285327,31.959895,42.4250,,,,,1
4,0.623917,31.959895,-65.9719,,,,,1
5,0.634346,71.842967,-120.6220,1,,,,
6,0.645621,115.818695,-180.6470,1,,,,
7,0.659898,171.809967,-256.4190,1,,,,
8,2.950621,31.959895,-47.8166,1,,,,
9,2.956995,71.842967,-78.4979,,1,,,


In [11]:
positions   = data  [0:1]
true_matrix = matrix[0:1]
pred_matrix = np.array([[
    [1, 0, 0, 0, 0,],
    [1, 0, 0, 0, 0,],
    [0, 1, 0, 0, 0,],
    [0, 1, 0, 0, 0,],
    [0, 0, 1, 0, 0,],
    [0, 0, 1, 0, 0,],
    [0, 0, 0, 1, 0,],
    [0, 0, 0, 1, 0,],
    [0, 0, 0, 0, 1,],
    [0, 0, 0, 0, 1,],
    [1, 0, 0, 0, 0,],
    [1, 0, 0, 0, 0,],
    [0, 1, 0, 0, 0,],
    [0, 1, 0, 0, 0,],
    [0, 0, 1, 0, 0,],
    [0, 0, 1, 0, 0,],
    [0, 0, 0, 1, 0,],
    [0, 0, 0, 1, 0,],
    [0, 0, 0, 0, 1,],
    [0, 0, 0, 0, 1,],
]])
evaluation  = F(positions, true_matrix, pred_matrix).round(2)
by_hand     = reg_loss(positions[0], true_matrix[0], pred_matrix[0], order) 
print("Tensor Evalution {}".format(evaluation))
print("By Hand Calculation {}".format(by_hand))
visuals.display_matrices(data[0], true_matrix[0], 2, order, 0, 0)
visuals.display_matrices(data[0], pred_matrix[0], 2, order, 0, 0)

Tensor Evalution [ 3.72]
By Hand Calculation 3.715990933736299


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,1,,,,
1,0.272315,115.818695,217.9330,1,,,,
2,0.278982,71.842967,126.6710,1,,,,
3,0.285327,31.959895,42.4250,1,,,,
4,0.623917,31.959895,-65.9719,,1,,,
5,0.634346,71.842967,-120.6220,,1,,,
6,0.645621,115.818695,-180.6470,,1,,,
7,0.659898,171.809967,-256.4190,,1,,,
8,2.950621,31.959895,-47.8166,,,1,,
9,2.956995,71.842967,-78.4979,,,1,,


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,1,,,,
1,0.272315,115.818695,217.9330,1,,,,
2,0.278982,71.842967,126.6710,,1,,,
3,0.285327,31.959895,42.4250,,1,,,
4,0.623917,31.959895,-65.9719,,,1,,
5,0.634346,71.842967,-120.6220,,,1,,
6,0.645621,115.818695,-180.6470,,,,1,
7,0.659898,171.809967,-256.4190,,,,1,
8,2.950621,31.959895,-47.8166,,,,,1
9,2.956995,71.842967,-78.4979,,,,,1


In [12]:
positions   = data  [6:7]
true_matrix = matrix[6:7]
pred_matrix = np.array([[
    [0.6, 0.4, 0, 0, 0,],
    [0.6, 0, 0, 0, 0.4,],
    [0.6, 0, 0.4, 0, 0,],
    [0.6, 0, 0, 0, 0.4,],
    [0, 0.7, 0, .3, 0,],
    [0, 0.7, 0, 0, .3,],
    [0, 0.7, 0, 0, .3,],
    [0, 0.7, 0, 0, .3,],
    [0, 0, 1, 0, 0,],
    [0, 0, 1, 0, 0,],
    [0, 0, 1, 0, 0,],
    [0.5, 0.2, 0.2, 0.1, 0,],
    [0, 0, 0, 1, 0,],
    [0, 0, 0, 1, 0,],
    [0, 0, 0, 1, 0,],
    [0, 0, 0, 1, 0,],
    [0, 0, 0, 0, 1,],
    [0, 0, 0, 0, 1,],
    [0, 0, 0, 0, 1,],
    [0, 0, 0, 0, 1,],
]])
evaluation = F(positions, true_matrix, pred_matrix).round(2)
by_hand = reg_loss(positions[0], true_matrix[0], pred_matrix[0], order) 
print("Tensor Evalution {}".format(evaluation))
print("By Hand Calculation {}".format(by_hand))
visuals.display_matrices(data[0], true_matrix[0], 2, order, 0, 0)
visuals.display_matrices(data[0], pred_matrix[0], 2, order, 0, 0)

Tensor Evalution [ 0.07]
By Hand Calculation 0.07424375863832218


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,1,,,,
1,0.272315,115.818695,217.9330,1,,,,
2,0.278982,71.842967,126.6710,1,,,,
3,0.285327,31.959895,42.4250,1,,,,
4,0.623917,31.959895,-65.9719,,1,,,
5,0.634346,71.842967,-120.6220,,1,,,
6,0.645621,115.818695,-180.6470,,1,,,
7,0.659898,171.809967,-256.4190,,1,,,
8,2.950621,31.959895,-47.8166,,,1,,
9,2.956995,71.842967,-78.4979,,,1,,


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,0.6,0.4,,,
1,0.272315,115.818695,217.9330,0.6,,,,0.4
2,0.278982,71.842967,126.6710,0.6,,0.4,,
3,0.285327,31.959895,42.4250,0.6,,,,0.4
4,0.623917,31.959895,-65.9719,,0.7,,0.3,
5,0.634346,71.842967,-120.6220,,0.7,,,0.3
6,0.645621,115.818695,-180.6470,,0.7,,,0.3
7,0.659898,171.809967,-256.4190,,0.7,,,0.3
8,2.950621,31.959895,-47.8166,,,1,,
9,2.956995,71.842967,-78.4979,,,1,,


In [13]:
positions   = data  [2:3]
true_matrix = matrix[2:3]
pred_matrix = np.array([[
    [.4, .3, .2, .1, 0,],
    [.4, .3, .2, .1, 0,],
    [.4, .3, .2, .1, 0,],
    [.4, .3, .2, .1, 0,],
    [0, .4, .3, .2, .1,],
    [0, .4, .3, .2, .1,],
    [0, .4, .3, .2, .1,],
    [0, .4, .3, .2, .1,],
    [.1, 0, .4, .3, .2,],
    [.1, 0, .4, .3, .2,],
    [.1, 0, .4, .3, .2,],
    [.1, 0, .4, .3, .2,],
    [.2, .1, 0, .4, .3,],
    [.2, .1, 0, .4, .3,],
    [.2, .1, 0, .4, .3,],
    [.2, .1, 0, .4, .3,],
    [0, .1, .2, .3, .4,],
    [0, .1, .2, .3, .4,],
    [0, .1, .2, .3, .4,],
    [0, .1, .2, .3, .4,],
]])
evaluation = F(positions, true_matrix, pred_matrix).round(2)
by_hand = reg_loss(positions[0], true_matrix[0], pred_matrix[0], order) 
print("Tensor Evalution {}".format(evaluation))
print("By Hand Calculation {}".format(by_hand))
visuals.display_matrices(data[0], true_matrix[0], 2, order, 0, 0)
visuals.display_matrices(data[0], pred_matrix[0], 2, order, 0, 0)

Tensor Evalution [ 0.]
By Hand Calculation 0.0


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,1,,,,
1,0.272315,115.818695,217.9330,1,,,,
2,0.278982,71.842967,126.6710,1,,,,
3,0.285327,31.959895,42.4250,1,,,,
4,0.623917,31.959895,-65.9719,,1,,,
5,0.634346,71.842967,-120.6220,,1,,,
6,0.645621,115.818695,-180.6470,,1,,,
7,0.659898,171.809967,-256.4190,,1,,,
8,2.950621,31.959895,-47.8166,,,1,,
9,2.956995,71.842967,-78.4979,,,1,,


,phi,r,z,A,B,C,D,E
0,0.263837,171.809967,331.9900,0.4,0.3,0.2,0.1,
1,0.272315,115.818695,217.9330,0.4,0.3,0.2,0.1,
2,0.278982,71.842967,126.6710,0.4,0.3,0.2,0.1,
3,0.285327,31.959895,42.4250,0.4,0.3,0.2,0.1,
4,0.623917,31.959895,-65.9719,,0.4,0.3,0.2,0.1
5,0.634346,71.842967,-120.6220,,0.4,0.3,0.2,0.1
6,0.645621,115.818695,-180.6470,,0.4,0.3,0.2,0.1
7,0.659898,171.809967,-256.4190,,0.4,0.3,0.2,0.1
8,2.950621,31.959895,-47.8166,0.1,,0.4,0.3,0.2
9,2.956995,71.842967,-78.4979,0.1,,0.4,0.3,0.2


In [14]:
np.random.seed(1010)
for event_id in range(50):
    positions = data[event_id:(event_id + 1)]
    true_matrix = matrix[event_id:(event_id + 1)]
    pred_matrix = np.array([
        [1, 0, 0, 0, 0,],
        [1, 0, 0, 0, 0,],
        [0, 1, 0, 0, 0,],
        [0, 1, 0, 0, 0,],
        [0, 0, 1, 0, 0,],
        [0, 0, 1, 0, 0,],
        [0, 0, 0, 1, 0,],
        [0, 0, 0, 1, 0,],
        [0, 0, 0, 0, 1,],
        [0, 0, 0, 0, 1,],
    ])
    pred_matrix = np.vstack([pred_matrix, pred_matrix])
    #pred_matrix = np.vstack([pred_matrix, np.random.rand(*pred_matrix.shape)])
    pred_matrix = pred_matrix / pred_matrix.sum(-1, keepdims=True)
    np.random.shuffle(pred_matrix)
    pred_matrix = np.reshape(pred_matrix, (1, *pred_matrix.shape))
    evaluation = F(positions, true_matrix, pred_matrix).round(2)[0]
    by_hand    = reg_loss(positions[0], true_matrix[0], pred_matrix[0], order).round(2)
    diff = np.round(evaluation - by_hand, 2)
    print("{3:>3}.) {0:>12} - {1:>12} = {2:>12}".format(evaluation, by_hand, diff, event_id))
    if not np.isfinite(diff):
        visuals.display_matrices(positions[0], pred_matrix[0], 2, order, 0, 0)
    #visuals.display_matrices(positions[0], true_matrix[0], 2, order, 0, 0)
    #visuals.display_matrices(positions[0], pred_matrix[0], 2, order, 0, 0)

  0.)         3.87 -         3.87 =          0.0
  1.)        93.25 -        93.25 =          0.0
  2.)        15.11 -        15.11 =          0.0
  3.)         0.97 -         0.97 =          0.0
  4.)         6.23 -         6.23 =          0.0
  5.)        96.37 -        96.37 =          0.0
  6.)          6.3 -          6.3 =          0.0
  7.)       128.48 -       128.48 =          0.0
  8.)          5.2 -          5.2 =          0.0
  9.)        62.57 -        62.57 =          0.0
 10.)         9.94 -         9.94 =          0.0
 11.)         4.74 -         4.74 =          0.0
 12.)         44.8 -         44.8 =          0.0
 13.)       116.39 -       116.39 =          0.0
 14.)        75.43 -        75.43 =          0.0
 15.)        14.73 -        14.73 =          0.0
 16.)        20.61 -        20.61 =          0.0
 17.)    230186.49 -    230186.98 =        -0.49
 18.)        50.14 -        50.14 =          0.0
 19.)         5.96 -         5.96 =          0.0
 20.)         21.0 -

In [15]:
np.random.seed(1010)
for event_id in range(50):
    positions = data[event_id:(event_id + 1)]
    true_matrix = matrix[event_id:(event_id + 1)]
    pred_matrix = np.array([
        [1, 0, 0, 0, 0,],
        [1, 0, 0, 0, 0,],
        [0, 1, 0, 0, 0,],
        [0, 1, 0, 0, 0,],
        [0, 0, 1, 0, 0,],
        [0, 0, 1, 0, 0,],
        [0, 0, 0, 1, 0,],
        [0, 0, 0, 1, 0,],
        [0, 0, 0, 0, 1,],
        [0, 0, 0, 0, 1,],
    ])
    pred_matrix = np.vstack([pred_matrix, np.random.rand(*pred_matrix.shape)])
    pred_matrix = pred_matrix / pred_matrix.sum(-1, keepdims=True)
    np.random.shuffle(pred_matrix)
    pred_matrix = np.reshape(pred_matrix, (1, *pred_matrix.shape))
    if event_id == 7:
        visuals.display_matrices(positions[0], pred_matrix[0], 2, order, 0, 0)
    evaluation = F(positions, true_matrix, pred_matrix).round(2)[0]
    by_hand    = reg_loss(positions[0], true_matrix[0], pred_matrix[0], order).round(2)
    diff = np.round(evaluation - by_hand, 2)
    print("{3:>3}.) {0:>12} - {1:>12} = {2:>12}".format(evaluation, by_hand, diff, event_id))

  0.)         2.52 -         2.43 =         0.09
  1.)        80.18 -        80.18 =          0.0
  2.)        19.26 -        19.06 =          0.2
  3.)         4.23 -         4.48 =        -0.25
  4.)         7.02 -         6.97 =         0.05
  5.)        96.57 -       101.96 =        -5.39
  6.)         7.92 -         6.34 =         1.58


,phi,r,z,A,B,C,D,E
0,0.085992,171.809967,-34.07190,0.426649,0.133553,0.10444,0.311954,0.0234041
1,0.097001,115.818695,-28.72810,,,1,,
2,0.105627,71.842967,-24.56560,0.263064,0.329213,0.00340367,0.12454,0.279779
3,0.113099,31.959895,-20.79690,1,,,,
4,0.835574,171.809967,-331.92200,,,,1,
5,0.844635,115.818695,-217.04400,,,,,1
6,0.851356,71.842967,-131.43800,,1,,,
7,0.857501,31.959895,-53.51270,0.0554037,0.0277254,0.288548,0.291276,0.337047
8,0.925332,31.959895,-23.98440,0.170139,0.304555,0.260601,0.259888,0.00481735
9,0.936337,71.842967,-31.57640,1,,,,


  7.)       130.76 -       129.67 =         1.09
  8.)         3.72 -         3.64 =         0.08
  9.)        63.66 -        63.46 =          0.2
 10.)        10.17 -         10.2 =        -0.03
 11.)         3.58 -         3.61 =        -0.03
 12.)         32.4 -        32.26 =         0.14
 13.)       148.53 -       141.76 =         6.77
 14.)      5209.04 -      5185.54 =         23.5
 15.)        14.81 -         19.5 =        -4.69
 16.)         21.3 -        21.22 =         0.08
 17.)    226167.14 -    229642.36 =     -3475.22
 18.)        49.82 -        49.82 =          0.0
 19.)         9.19 -         9.72 =        -0.53
 20.)        15.93 -        16.11 =        -0.18
 21.)        33.38 -        33.38 =          0.0
 22.)        61.46 -        61.23 =         0.23
 23.)         6.64 -         6.75 =        -0.11
 24.)      1808.84 -      1867.78 =       -58.94
 25.)        34.97 -        40.96 =        -5.99
 26.)          2.8 -         2.73 =         0.07
 27.)         5.72 -